In [ ]:
# Importando os pacotes necessários
import pandas as pd

In [ ]:
# Comandos auxiliares
from IPython.core.display import display, HTML
pd.set_option('display.max_columns', 6)

def display_side_by_side(dfs: list, captions: list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes(
            "style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

# Integração de Dados

---

Até agora, nós realizamos a **limpeza** do nosso conjunto de dados, garantindo a remoção de possíveis impurezas. 

Agora, a próxima etapa da *Preparação* é **combinar dados** de diferentes fontes para obter uma estrutura unificada com informações mais significativas. 

De fato, é bastante comum em aplicações da vida real a necessidade de extrair informação a partir de dados de diferentes fontes. 

Por exemplo, imagine que você precise analisar a popularidade de uma lista de músicas. No entanto, você possui apenas uma tabela separada listando informações básicas de todas as músicas presentes nos *Charts* do Spotify, incluindo o nome e identificador das músicas e de seus artistas. 

Portanto, para obter as informações de popularidade musical, você decide coletar informações extras dos *Charts*, onde você obtém uma tabela contendo o *score* da popularidade e a data de lançamento das músicas. 

Porém, nesta tabela, a única informação de identificação da música é o seu **id**. 
Consequentemente, apesar de ter as informações que você precisava, sem o nome seria difícil dizer qual música você está analisando. 

> **O que fazer então?** 🤔

Se quisermos fazer uma análise completa sobre a popularidade das músicas coletadas, precisamos reunir os dados das duas tabelas em um único lugar. 

➡️ **Esse processo é denominado <ins>Integração de dados</ins>**

Nesta seção, nós iremos explorar um recurso essencial oferecido pelo *pandas* para realizar **integração de dados**: a função `merge()`.

## `merge()`
---

Muito utilizada para fazer operações de junção comumente aplicadas em banco de dados.
Especificamente, quando você deseja combinar objetos de dados com base em uma ou mais **colunas-chave** de maneira semelhante a um banco de dados relacional, `merge()` é a ferramenta ideal. 

Essa função implementa vários tipos de associações: 

- um-para-um (1:1)
- um-para-muitos (1:N) 
- muitos-para-muitos (N:N)

O tipo de associação realizada depende **essencialmente** da organização dos conjuntos de dados de entrada. 

Aqui, nós mostraremos exemplos simples destes três tipos de associação e discutiremos opções detalhadas mais adiante.

### EXEMPLO

Inicialmente, para exemplificar, nós iremos **integrar** informações sobre as músicas do nosso conjunto de dados a partir de diferentes tabelas. 

Para isso, nós dividimos a tabela `Tracks` em dois *DataFrames*, `df1` e `df2`.  

### `df1`
- identificador da música: `song_id`
- título da música: `song_name`
- identificador do(s) artista(s): `artist_id`
- nome do(s) artista(s): `artist_name`

### `df2`
- identificador da música: `song_id`
- popularidade da música: `popularity`
- data de lançamento: `release_date`

**ATENÇÃO** ⚠️
> Ambas as tabelas possuem a coluna `song_id` para identificar cada uma das músicas.

In [ ]:
# Lendo os dados e criando os dois DataFrames
df1 = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv',
    usecols=['song_id', 'song_name', 'artist_id', 'artist_name'], # seleciona campos específicos
    encoding='utf-8')
df2 = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv',
    usecols=['song_id', 'popularity', 'release_date'], # seleciona campos específicos
    encoding='utf-8')

display_side_by_side([df1.head(3), df2.head(3)], ['df1', 'df2']) # imprime as 3 primeiras linhas

### Um-para-um (1:1) 

A associação **um-para-um** é talvez o tipo mais simples de fusão, muito semelhante à concatenação de colunas. 

Neste tipo de relacionamento, cada registro na primeira tabela pode ter **somente um** registro correspondente na segunda tabela, e cada registro na segunda tabela pode ter **somente um** registro correspondente na primeira tabela (como mostrado na figura a seguir).


<img src="figure1.png" alt="Junção 1:1" style="width: 500px;" class="center"/>

Neste exemplo, nós unimos as tabelas `df1` e `df2` para reunir informações de popularidade das músicas do Spotify usando a função `merge()`.

Ao usar `merge()`, você fornecerá dois argumentos obrigatórios:

* `left`: *DataFrame* esquerdo
* `right`: *DataFrame* direito

No nosso caso, nós passamos as tabelas `df1`e `df2`.

Depois disso, você pode fornecer vários argumentos opcionais para definir como as tabelas serão mescladas. 
Neste primeiro exemplo, o único parâmetro que precisamos setar é:

* `on`: informa quais colunas ou índices (i.e., **colunas-chave** ou **índices-chave**) você deseja unir
    
No nosso caso, passaremos a coluna `song_id`, que está presente em ambas as tabelas, como vimos anteriormente.

In [ ]:
# Associação um-para-um
df3 = pd.merge(left=df1, right=df2, on='song_id')
df3.head(3)

<img src="figure1_result.png" alt="Junção 1:1" style="width: 700px;" class="center"/>

O resultado da junção é um único *DataFrame* que combina as informações das duas entradas, baseado nos valores comuns presentes na coluna `song_id` de ambos os *DataFrames* originais.

### OBSERVAÇÕES

* Lembre-se que se você usar o parâmetro `on`, a coluna ou índice especificado deve estar presente nas duas tabelas.

* Se você não especificar o parâmetro `on`, as colunas que compartilham o mesmo nome nos dois *DataFrames* serão usadas como **colunas-chave** na junção. 

> **Vamos testar?** 😃

In [ ]:
# Associação um-para-um
df3 = pd.merge(left=df1, right=df2) # SEM ESPECIFICAR A COLUNA-CHAVE
df3.head(3)

### Um-para-muitos (1:N)

A associação **um-para-muitos** é usada quando uma das duas **colunas-chave** contém mais de uma entrada por registro (como mostrado na figura a seguir).

<img src="figure2.png" alt="Junção 1:N" style="width: 500px;" class="center"/>

Para exemplificar, vamos realizar a junção da tabela resultante do exemplo anterior (i.e., *DataFrame* `df3`) com os sucessos do Spotify na tabela `Charts` (i.e., *DataFrame* `df4`). 

In [ ]:
# Lendo os dados
df4 = pd.read_table('../dataset/spotify_charts_complete.tsv',
    usecols=['chart_week', 'song_id', 'position'], # seleciona campos específicos
    encoding='utf-8')
df4 = df4.sort_values(by=['song_id']) # ordena o DataFrame pelo id da música

display_side_by_side([df3.head(3), df4.head(3)], ['df3', 'df4']) # imprime as 3 primeiras linhas

⚠️ **Note que, nesta tabela, uma música pode aparecer em qualquer semana dos *Charts* do Spotify.** Ou seja, elas podem repetir!

Portanto, para **qualquer** música representada na tabela `df3`, pode haver **mais de uma** ocorrência na tabela `df4`. 

Neste caso, a associação entre a tabela `df3` e a tabela `df4` é uma relação **um-para-muitos**.

> **Vamos integrá-las?**

Assim como no exemplo anterior, nós passaremos os dois *DataFrames* para a função `merge()`, bem como a **coluna-chave** `song_id` presente em ambas as tabelas:

In [ ]:
# Associação um-para-muitos de dois DataFrames
df5 = pd.merge(left=df3, right=df4, on='song_id') 
df5.head(3)

<img src="figure2_result.png" alt="Junção 1:N" style="width: 800px;" class="center"/>

O resultado da junção é um único *DataFrame* que combina as informações das duas entradas; **porém**, ao contrário do exemplo anterior, as informações originais (i.e., do *DataFrame* `df3`) se repetem conforme exigido pelas entradas do *DataFrame* `df4`.

### Muitos-para-muitos (N:N)

Um relacionamento **muitos-para-muitos** ocorre quando vários registros em uma tabela são associados a vários registros em outra tabela (como mostrado na figura a seguir). 

<img src="figure3.png" alt="Junção N:N" style="width: 500px;" class="center"/>

Para exemplificar, suponha que você precisa integrar as várias músicas da tabela resultante anterior (i.e., *DataFrame* `df5`) com outra tabela contendo o número total de *streams* que cada música atingiu nas semanas dos *Charts* (i.e., *DataFrame* `df6`). 

⚠️ **Observe** que em ambas as tabelas a **coluna-chave** `song_id` possui entradas repetidas. 

Quando isso acontece, ou seja, se a **coluna-chave** das duas tabelas incluir entradas duplicadas, o resultado é uma fusão **muitos-para-muitos**. 

In [ ]:
# Lendo os dados
df6 = pd.read_table('../dataset/spotify_charts_complete.tsv',
    usecols=['chart_week', 'song_id', 'streams'], # seleciona campos específicos
    encoding='utf-8')
df6 = df6.sort_values(by=['song_id']) # ordena o DataFrame pelo id da música

display_side_by_side([df5.head(3), df6.head(3)], ['df5', 'df6']) # imprime as 3 primeiras linhas

Antes de prosseguirmos com a junção, note que as duas tabelas possuem em comum, além da coluna `song_id`, a coluna `chart_week`. Ou seja, neste exemplo, teremos duas **colunas-chave** para realizar a junção. 

In [ ]:
# Associação muitos-para-muitos de dois DataFrames
df7 = pd.merge(left=df5, right=df6, on=['song_id', 'chart_week'])
df7.head(3)

<img src="figure3_result.png" alt="Junção N:N" style="width: 800px;" class="center"/>

Após a junção **muitos-para-muitos**, a **coluna-chave** do *DataFrame* resultante será a combinação das **colunas-chave** de cada tabela. 

---
## VERIFICAÇÃO

Além dos parâmetros vistos anteriormente, a função `merge()` possui o argumento `validate` para verificar se a junção realizada é do tipo especificado. Os tipos disponíveis são:

* `one_to_one` ou `1:1`: verifica se as **chaves** são exclusivas nos conjuntos de dados esquerdo e direito.
* `one_to_many` ou `1:m`: verifica se as **chaves** são exclusivas no conjunto de dados esquerdo.
* `many_to_one` ou `m:1`: verifica se as **chaves** são exclusivas no conjunto de dados correto.
* `many_to_many` ou `m:m`: permitido, mas não resulta em verificações.

> **Vamos verificar se as integrações realizadas anteriormente estão corretas!** 👀

In [ ]:
# Associação um-para-um
df3 = pd.merge(left=df1, right=df2, on='song_id', validate='one_to_one') # verificando
df3.head(3)

In [ ]:
# Associação um-para-muitos
df5 = pd.merge(left=df3, right=df4, on='song_id', validate='one_to_many') # verificando
df5.head(3)

In [ ]:
# Associação muitos-para-muitos
df7 = pd.merge(left=df5, right=df6, on=['song_id','chart_week'], validate='many_to_many') # verificando
df7.head(3)

**SIM!** Está tudo correto! ✔️

Se o tipo das associações das junções anteriores estivessem incorretas, ou seja, se a validação falhasse, a função geraria um `MergeError`.

> **Vamos fazer um exemplo teste?**

Vamos tentar unir as tabelas `df4` e `df5`, que possuem **chaves** duplicadas da **coluna-chave** `song_id`, utilizando a associação **um-para-muitos**.

In [ ]:
# Testando uma validação incorreta
pd.merge(left=df4, right=df5, on='song_id', validate='one_to_many') # verificando

Como esperado, a função lançou um `MergeError` indicando que o *DataFrame* da esquerda possui **chaves** duplicadas, o que não é aceitável em uma associação do tipo **um-para-muitos**.

---
## TIPOS DE JUNÇÕES

Além das associações, a função `merge()` disponibiliza quatro tipos de junções:

* `inner`: o tipo de junção *default* da função `merge()`, e se baseia na **interseção** de **chaves** de ambos os *DataFrames*
* `outer`: se baseia na **união** de **chaves** do *DataFrame* esquerdo e os registros correspondentes do *DataFrame* direito
* `left`: usa apenas as **chaves** do *DataFrame* esquerdo
* `right`: usa apenas as **chaves** do *DataFrame* direito

<br>
<img src="figure4.png" alt="tipos de junção" style="width: 700px;" class="center"/>
<br>

### EXEMPLO

Para entendermos melhor cada tipo de junção, vamos utilizar duas tabelas do nosso conjunto de dados nos exemplos a seguir.

In [ ]:
# Lendo os dados
charts = pd.read_table('../dataset/spotify_charts_complete.tsv',
    usecols=['song_id', 'streams', 'position'], # seleciona campos específicos
    encoding='utf-8')
tracks = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv',
    usecols=['song_id', 'song_name', 'artist_id', 'popularity', 'song_type'], # seleciona campos específicos
    encoding='utf-8')

display_side_by_side([charts.head(3), tracks.head(3)], ['Charts', 'Tracks'])

#### `inner`

Neste exemplo, vamos realizar um `inner join` entre a tabela `Tracks` e a tabela que armazena as músicas **#1** dos *Charts* do Spotify utilizando a função `merge()` com seus argumentos *default*.

Para isso, vamos criar a segunda tabela primeiro:

In [ ]:
# Criando a tabela #1 Hits
top_one_hits = charts[charts['position'] == 1] # selecionando apenas as músicas com posição 1
top_one_hits.head()

Agora, podemos prosseguir para a junção:

In [ ]:
# inner join, utilizando os parâmetros default
inner_merged = pd.merge(tracks, top_one_hits) 
inner_merged.head()

Como resultado, a junção mantém apenas as linhas que estão presentes na **coluna-chave** dos dois *DataFrames*.

Para verificar se a junção deu certo, dado que todas as músicas da tabela `Tracks` estão presentes da tabela `Charts`, nós podemos comparar o número total de linhas do *DataFrame* resultante (`inner_merged`) com o número total de linhas do menor *DataFrame* da junção. 

In [ ]:
# Calculando o tamanho das tabelas usadas na junção
print(len(tracks))
print(len(top_one_hits))

No nosso caso, a tabela `top_one_hits` é a menor. Portanto, o *DataFrame* resultante precisa ter o mesmo número de linhas:

In [ ]:
# Verificando se o tamanho das tabelas são iguais
if (len(inner_merged) == len(top_one_hits)):
    print("✔️")
else:
    print("❌")

#### `outer`

Neste exemplo, vamos realizar um `outer join` entre a tabela `Charts` e a tabela `top_one_hits` utilizando a função `merge()` com o parâmetro `how`:

In [ ]:
# Visualizando as duas tabelas
display_side_by_side([charts.head(3), top_one_hits.head(3)], ['Charts', '#1 Hits'])

In [ ]:
# outer join
outer_merged = pd.merge(charts, top_one_hits, how="outer")
outer_merged.head()

⚠️ **OBSERVAÇÃO!** Como foi mostrado na figura anterior, em um `outer join`, todas as linhas de ambos os *DataFrames* estarão presentes no *DataFrame* final. Portanto, neste exemplo, após a junção, teríamos que ter exatamente as mesmas linhas da tabela `Charts`, já que a tabela `top_one_hits` é um subconjunto da mesma.

Utilizando a função `equals` do *pandas*, podemos verificar se os dois *DataFrames* contêm os mesmos elementos:

In [ ]:
# Verificando se dois objetos contêm os mesmos elementos
if (charts.equals(outer_merged)):
    print("✔️")
else:
    print("❌")

#### `left` / `right`

Neste exemplo, vamos realizar apenas um `left join`, dado que um `right join` seguiria os mesmo passos, porém considerando o *DataFrame da direita*.

Aqui, nós vamos realizar a junção entre a tabela `top_one_hits` e a tabela `Tracks` utilizando a função `merge()` com o parâmetro `how`:

In [ ]:
# Visualizando as duas tabelas
display_side_by_side([top_one_hits.head(3), tracks.head(3)], ['#1 Hits', 'Tracks'])

In [ ]:
# left join
left_merged = pd.merge(top_one_hits, tracks, how="left")
left_merged.head()

O resultado da junção é um *DataFrame* contendo todos os **#1** *hits* dos *Charts* do Spotify e suas informações básicas.

A partir desse resultado, poderíamos, por exemplo, verificar o intervalo do *score* de popularidade de tais músicas:

In [ ]:
left_merged.popularity.value_counts() # verificando os scores de popularidade únicos

In [ ]:
left_merged.popularity.mean() # verificando a média das popularidades

Podemos ver que, no geral, a popularidade de tais músicas é bem alta, com uma média de 82! 

### Conclusão

Este notebook apresentou como fazer a integração de dados de diferentes fontes aplicando a função `merge()` da biblioteca *pandas*. 

🔎 **Se interessou?** Dê uma olhada na documentação do *pandas* para informações extras sobre integração de dados:
[Merging with *pandas*](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

---

O próximo notebook ([4.2.Transformacao.ipynb](4.2.Transformacao.ipynb)) apresenta como transformar dados de diferentes formatos em formatos mais amigáveis para modelos e algoritmos de Aprendizado de Máquina.